In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 22.1 MB/s 
     |████████████████████████████████| 7.6 MB 51.1 MB/s 
     |████████████████████████████████| 163 kB 56.5 MB/s 


In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    ["I've been waiting for a HuggingFace course my whole life.",
     "I hate this so much!",
     ]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

#1. Pipeline 내부 실행 과정

## 토크나이저를 이용한 전처리

토크나이저의 역할

1.   입력을 token이라 부를 수 있는 word, subword 또는 symbol로 분할
2.   각 token을 integer mapping
3.   모델에 유용할 수 있는 additional inputs 추가

* Architecture: 모델의 뼈대를 의미, 모델 내에서 발생하는 각 레이어(layer)와 오퍼레이션(operation)등을 정의
* Checkpoint: 해당 architecture에서 로드될 가중치 값들


In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [4]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
    ]

# return_tensors는 토크나이저가 반환하는 텐서의 유형을 지정하는 파라미터로 PyTorch, TensorFlow, Numpy등으로 지정할 수 있다.
inputs = tokenizer(raw_inputs, padding = True, truncation = True, return_tensors = "pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


## 모델 살펴보기

In [5]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

# 이전과 동일한 체크포인트를 다운로드하고 모델을 초기화

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 고차원 벡터?

Transformer 모듈의 벡터 출력은 일반적으로 3가지 차원으로 이루어짐


*   Batch Size: 한 번에 처리되는 sequence의 수 (위의 예제: 2)
*   Sequence Length: sequence 숫자 표현의 길이 (위의 예제: 16)
*   Hidden size: 각 모델 입력의 벡터 차원

Hidden size가 일반적으로 굉장히 크기 때문에 고차원 벡터라고 나타남  
작은 모델은 일반적으로 768, 큰 모델은 3072 이상일 수도 있음  


In [6]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


## Model heads: 숫자 이해하기
  
Model head는 hidden states의 고차원 벡터를 입력으로 받아 다른 차원에 projection  
일반적으로 하나, 혹은 몇 개의 linear layers로 구성

In [10]:
# 해당 섹션의 예시에서는 sequence classification head가 포함되어 있는 모델이 필요하기 때문에
# AutoModel이 아닌 sequence classfication head가 포함되어있는 AutoModelForSequenceClassification 사용
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([2, 2])


두 개의 문장과 두 개의 레이블만 있기 때문에 2 x 2의 shape이 출력됨  
기존의 2 x 16 x 768보다 차원이 훨씬 낮아진 것을 확인할 수 있음

## 출력 후처리하기

In [11]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


모델의 output이 항상 의미 있는 값은 아닌데, 위의 예시에서도 단순한 logit 점수를 출력하고 있다.  
이를 확률값으로 변환하기 위해서는 softmax 계층을 통과해야한다.  
모든 Transformers의 loss function은 일반적으로 최종 activation function으로 softmax를, 손실함수로는 cross entropy를 사용하여 구현되기 때문이다.

In [12]:
# 문장별 특정 레이블에 속할 확률
import torch
predictions = torch.nn.functional.softmax(outputs.logits, dim = -1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [13]:
# 각 레이블이 무엇인지 확인하기 위함
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

## 혼자 해보기

In [20]:
classifier = pipeline("sentiment-analysis")
classifier(
    ["I hate cucumber.",
     "I love listening to music.",
     ]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9956943988800049},
 {'label': 'POSITIVE', 'score': 0.999747097492218}]

In [21]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [22]:
raw_inputs = [
    "I hate cucumber.",
    "I love listening to music.",
    ]

# return_tensors는 토크나이저가 반환하는 텐서의 유형을 지정하는 파라미터로 PyTorch, TensorFlow, Numpy등으로 지정할 수 있다.
inputs = tokenizer(raw_inputs, padding = True, truncation = True, return_tensors = "pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  5223, 12731, 24894,  5677,  1012,   102],
        [  101,  1045,  2293,  5962,  2000,  2189,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


In [23]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim = -1)
print(model.config.id2label)
print(predictions)

{0: 'NEGATIVE', 1: 'POSITIVE'}
tensor([[9.9569e-01, 4.3055e-03],
        [2.5286e-04, 9.9975e-01]], grad_fn=<SoftmaxBackward0>)


# 2. Models

## 트랜스포머 모델 생성하기

In [24]:
from transformers import BertConfig, BertModel

# BERT모델을 초기화하기 위해서는 가장 먼저 configurateion(설정) 객체를 로드해야함
# config(설정) 만들기
config = BertConfig()

# 해당 config에서 모델 생성
model = BertModel(config)

In [25]:
# configuration 객체에는 모델을 빌드하는데 필요한 많은 속성이 포함되어 있음
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



## 다른 loading methods

위의 모델은 학습되지 않은 모델로 당연히 성능이 좋지 않다  
따라서 먼저 학습이 진행되어야하는데, 이보단 pretrained model을 사용하는 것이 현명

In [26]:
model = BertModel.from_pretrained("bert-base-cased")
# BertConfig를 사용하지 않고 bert-base-cased를 통해 사전 학습된 모델 로드

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Saving methods

In [29]:
model.save_pretrained("saving_folder")

In [30]:
ls saving_folder

config.json  pytorch_model.bin


## 트랜스포머 모델을 활용한 추론

In [31]:
sequences = ["Hello!", "Cool.", "Nice!"]

In [32]:
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 99, 102],
]

In [33]:
# array를 tensor로 변환
model_inputs = torch.tensor(encoded_sequences)

## 모델의 입력으로 텐서 활용

In [35]:
output = model(model_inputs)

# 3. 토크나이저
입력된 텍스트를 모델에서 처리할 수 있는 데이터로 변환  
모델은 숫자만 처리할 수 있으므로, 토크나이저는 텍스트 입력을 숫자 데이터로 변환

## 단어 기반 토큰화(Word-based Tokenization)

vocabulary에 없는 단어를 표현하기 위해서는 unknown tokekn([UNK])와 같은 사용자 정의 토큰이 필요한데, unknown token이 많이 등장하는 것은 토크나이저가 해당 단어의 합당한 표현을 찾을 수 없고, 그 과정에서 정보를 많이 잃어버린다는 뜻이므로 나쁜 징조이다. 따라서 이런 unknown token이 최대한 적게 출력되게끔 하는 것이 목표이고, 이를 위한 한 가지 방법으로 문자 기반 토크나이저를 사용하는 것이 있다.

In [36]:
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

['Jim', 'Henson', 'was', 'a', 'puppeteer']


## 문자 기반 토큰화(Character-based Tokenization)
vocabulary의 크기가 단어 기반 토큰화보다 훨씬 작다.  
모든 단어들이 문자를 가지고 만들어질 수 있기 때문에, OOV(out-of-vocabulary) 토큰이 훨씬 적다.  
그러나 분리된 토큰 표현이 문자이기 때문에 토큰의 의미 파악이 어렵다. 
모델에서 처리해야하는 양의 토큰이 매우 많아진다.  
단어 기반 토큰화에서는 하나의 토큰으로 표현되던 하나의 단어가 문자 기반 토큰화에서는 스무 개의 토큰이 될 수도 있다.  
단어 기반 토큰화와 문자 기반 토큰화의 장점을 결합한 하위 단어 토큰화(subword tokenization)을 사용할 수 있다.

## 하위 단어 토큰화(Subword Tokenization)
빈번하게 사용되는 단어에 대해서는 subwords로 분할하지 않고, 희귀 단어에 대해서는 의미있는 subwords로 분할한다.  
예를 들어 annoyingly를 annoying+ly로 분해해 annoying과 ly의 합성의미로 유지

## 토크나이저 로딩 및 저장

In [37]:
# Bert와 동일한 checkpoint로 학습된 bert tokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [38]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [39]:
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [40]:
# 모델 저장과 동일하게 토크나이저를 저장할 수 있음
tokenizer.save_pretrained("saving_folder")

('saving_folder/tokenizer_config.json',
 'saving_folder/special_tokens_map.json',
 'saving_folder/vocab.txt',
 'saving_folder/added_tokens.json',
 'saving_folder/tokenizer.json')

## Encoding
텍스트를 숫자로 변환하는 과정  
토큰화와 입력 식별자(숫자)로의 변환의 2단계 프로세스를 수행
토큰화: 텍스트를 토큰으로 분리  
입력 식별자로의 전환: 토큰들을 숫자로 변환하여 tensor로 만들어 이를 모델에 입력할 수 있도록 함


### 토큰화 작업

In [41]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


### 토큰을 입력 식별자로 변환(From tokens to input IDs)

In [43]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


## Decoding
입력 식별자를 이용해서 어휘집에서 해당 문자열을 찾음
인덱스를 다시 토큰으로 변환활 뿐 아니라 하위 단어로 분할된 토큰을 병합하여 읽을 수 있는 원본 문장을 도출  




In [45]:
decoded_string = tokenizer.decode(ids)
print(decoded_string)

Using a Transformer network is simple


#4. 다중 시퀀스 처리

In [51]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor([ids])
# This line will fail
# transformers모델은 기본적으로 다중 문장 시퀀스를 입력받기를 기대하기 때문에 단일 시퀀스를 입력하자 오류가 발생한 것
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


텐서의 형태는 항상 직사각형 모양이어야 하기 때문에 입력식별자 리스트를 같은 길이로 맞춰주기 위해 padding을 해야한다.

## Input Padding하기

In [53]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]
# tokenizer.pad_token_id에는 padding token의 식별자(ID)가 지정되어 있음

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


마지막 input의 두번째 row는 결과가 두번째와 같아야 하는데 다르다. 이는 attention layer가 시퀀스의 모든 토큰에 집중하면서 패딩 토큰에도 집중하기 때문인데, 따라서 attention layer가 패딩 토큰을 무시하도록 지시해야한다.

## Attention Mask

In [56]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask = torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


결과가 같아졌다!

## 길이가 더 긴 시퀀스들

Transformer는 모델에 입력할 수 있는 시퀀스의 길이에 제한이 있는데, 대부분의 모델이 최대 512/1024개 토큰 시퀀스를 처리하며, 그보다 긴 시퀀스의 처리를 요구 받으면 오류를 발생시킨다.  
이에 대한 솔루션으로는 

* 길이가 더 긴 시퀀스를 지원하는 모델 사용
* Truncation(시퀀스를 절단하기)
가 있다. 

In [57]:
max_sequence_length = 512
sequence = sequence[:max_sequence_length]

## Using Transformer API

In [60]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 단일 시퀀스 토큰화
sequence = "I've been waiting for a HugginFace course my whole life."
model_inputs = tokenizer(sequence)
## 다중 시퀀스 토큰화
sequences = ["I've been waiting for a HugginFace course my whole life.", "So have I!"]
model_inputs = tokenizer(sequences)

In [62]:
# 해당 시퀀스를 리스트 내 최장 시퀀스 길이까지 padding
model_inputs = tokenizer(sequences, padding = "longest")
print(model_inputs)
# 해당 시퀀스를 모델 최대 길이까지 padding
model_inputs = tokenizer(sequences, padding = "max_length")
print(model_inputs)
# 지정된 최대 길이까지 시퀀스를 padding
model_inputs = tokenizer(sequences, padding = "max_length", max_length = 8)
print(model_inputs)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 8549, 11528, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}
{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 8549, 11528, 12172, 2607, 2026, 2878, 2166, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

자동으로 어텐션 마스킹까지 된 것을 확인할 수 있다.

In [64]:
sequences = ["I've been waiting for a HugginFace course my whole life.", "So have I!"]

# 모델 최대 길이보다 긴 시퀀스 truncate
model_inputs = tokenizer(sequences, truncation = True)
print(model_inputs)

# 지정된 최대 길이보다 긴 시퀀스 truncate
model_inputs = tokenizer(sequences, max_length = 8, truncation = True)
print(model_inputs)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 8549, 11528, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}
{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


## 특수 토큰들

In [66]:
sequence = "I've been waiting for a HugginFace course my whole life."
model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 8549, 11528, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 8549, 11528, 12172, 2607, 2026, 2878, 2166, 1012]


앞 뒤로 id가 하나씩 추가된 것을 확인할 수 있다. 이를 디코딩하여 각각 무엇을 나타내는지 알아보자.

In [68]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] i've been waiting for a hugginface course my whole life. [SEP]
i've been waiting for a hugginface course my whole life.


시작 부분에 [CLS]가, 끝 부분에 [SEP]가 추가된 것을 확인할 수 있다.  
이는 모델이 해당 특수 토큰들로 pre-train됐기 때문이다. 

## 마무리: 토크나이저에서 모델로...
주요 API를 이용하여 다중 시퀀스, 매우 킨 시퀀스, 여러 유형의 텐서를 처리하기

In [70]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequences = ["I've been waiting for a HugginFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding = True, truncation = True, return_tensors = "pt")
# padding = true: 해당 시퀀스 리스트 내 최대 시퀀스 길이까지 padding
# truncation = true: 모델의 최대 길이보다 긴 시퀀스 truncate, 512 for distilbert
output = model(**tokens)
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.5268,  2.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
